Загрузка напрямую из документации GigaChat API

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
import re
from typing import Optional
from bs4 import BeautifulSoup, SoupStrainer
from langchain_community.document_loaders.sitemap import SitemapLoader


def metadata_extractor(
    meta: dict, soup: BeautifulSoup, title_suffix: Optional[str] = None
) -> dict:
    title_element = soup.find("title")
    description_element = soup.find("meta", attrs={"name": "description"})
    html_element = soup.find("html")
    title = title_element.get_text() if title_element else ""
    if title_suffix is not None:
        title += title_suffix

    return {
        "source": meta["loc"],
        "title": title,
        "description": description_element.get("content", "")
        if description_element
        else "",
        "language": html_element.get("lang", "") if html_element else "",
        **meta,
    }

def simple_extractor(html: str | BeautifulSoup) -> str:
    if isinstance(html, str):
        soup = BeautifulSoup(html, "lxml")
    elif isinstance(html, BeautifulSoup):
        soup = html
    else:
        raise ValueError(
            "Input should be either BeautifulSoup object or an HTML string"
        )
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()

def load_gigachat_docs():
    return SitemapLoader(
        "https://developers.sber.ru/docs/sitemap.xml",
        filter_urls=[r".*/gigachat/.*"],
        parsing_function=simple_extractor,
        default_parser="lxml",
        bs_kwargs={"parse_only": SoupStrainer(name=("article", "title"))},
        meta_function=lambda meta, soup: metadata_extractor(
            meta, soup, title_suffix=" | GigaChat"
        ),
    ).load()


def load_langchain_gigachat_docs():
    return SitemapLoader(
        "https://developers.sber.ru/docs/sitemap.xml",
        filter_urls=[r".*/gigachain/.*"],
        parsing_function=simple_extractor,
        default_parser="lxml",
        bs_kwargs={"parse_only": SoupStrainer(name=("article", "title"))},
        meta_function=lambda meta, soup: metadata_extractor(
            meta, soup, title_suffix=" | langchain-gigachat"
        ),
    ).load()

In [18]:
gigachat_docs = load_gigachat_docs()
gigachat_docs

Fetching pages: 100%|##########| 58/58 [00:17<00:00,  3.29it/s]


[Document(metadata={'source': 'https://developers.sber.ru/docs/ru/gigachat/about', 'title': 'Возможности GigaChat | Документация для разработчиков | GigaChat', 'description': '', 'language': '', 'loc': 'https://developers.sber.ru/docs/ru/gigachat/about', 'changefreq': 'weekly', 'priority': '0.5'}, page_content='Возможности GigaChat | Документация для разработчиковЭто полезныйматериал?Это полезный материал?Возможности GigaChatОбновлено 6 мая 2024GigaChat — это сервис, который умеет взаимодействовать с пользователем в формате диалога, писать код, создавать тексты и картинки по запросу пользователя. При этом GigaChat стремится избегать спорных этических вопросов или провокаций.GigaChat поддерживает русский и английский языки.Особенности и преимущества\ufeffОбучение модели\ufeffКорпус данных, использованных для обучения включает в себя книги и новости на русском и английском языках, разговорную речь, научные статьи и другие домены. Объем данных, использованных для обучения, составил 300 Гб

In [19]:
gigachain_docs = load_langchain_gigachat_docs()
gigachain_docs

Fetching pages: 100%|##########| 22/22 [00:09<00:00,  2.26it/s]


[Document(metadata={'source': 'https://developers.sber.ru/docs/ru/gigachain/concepts/architecture', 'title': 'Архитектура | Документация для разработчиков | langchain-gigachat', 'description': '', 'language': '', 'loc': 'https://developers.sber.ru/docs/ru/gigachain/concepts/architecture', 'changefreq': 'weekly', 'priority': '0.5'}, page_content='Архитектура | Документация для разработчиковЭто полезныйматериал?Это полезный материал?АрхитектураОбновлено 4 сентября 2024Фреймворк GigaChain состоит из нескольких пакетов.gigachain-core\ufeffПакет содержит базовые абстракции различных компонентов и способы их объединения.\nЗдесь определены интерфейсы для основных компонентов: LLM, векторных хранилищ, ретриверов и других.\nВ пакете нет интеграций со сторонними продуктами.\nЗависимости намеренно сведены к минимуму.Связанные пакеты\ufeffПолный список интеграций содержится в gigachain-community, тогда как интеграции с популярными сервисами выделены в собственные пакеты. Например, gigachain-openai

In [20]:
all_docs = []
all_docs.extend(gigachat_docs)
all_docs.extend(gigachain_docs)
len(all_docs)

80

In [21]:
from langchain.text_splitter import MarkdownTextSplitter

text_splitter = MarkdownTextSplitter(chunk_size=2000, chunk_overlap=500)
doc_splits = text_splitter.split_documents(all_docs)
print(f"Documents splited. Count: {len(doc_splits)}")

Documents splited. Count: 324


In [22]:
import os
import tqdm
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [27]:
import time

index_name = os.environ.get("PINECONE_INDEX_NAME", "gigachain-test-gigar-newdb")
pc.create_index(
    name=index_name,
    dimension=2560,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
while not pc.describe_index(index_name).status["ready"]:
    time.sleep(1)
    
index = pc.Index(index_name)

In [28]:
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
embeddings = GigaChatEmbeddings(model="EmbeddingsGigaR")
# from langchain_openai.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [29]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## Загрузка

In [30]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(doc_splits))]
vector_store.add_documents(documents=doc_splits, ids=uuids)
retriever = vector_store.as_retriever()
print("OK")

OK
